In [ ]:
import json
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import gamma
from scipy.special import gamma as gamma_fn
import re
import os
from matplotlib import pyplot as plt
from typing import Tuple, List

from distutils.dir_util import copy_tree

In [ ]:
from run_config_json import create_run_config_json

In [ ]:
STATE_NAMES_KEY = "stateTimeMap"
TOTAL_TIME_KEY = "t_total"

In [ ]:
# General settings
num_runs = 1
starting_seed = 0
seed_multiplier = 100

# Validator settings
num_nodes = 16
num_consensus = 3000
base_time_limit = 10000
node_processing_distribution = "exp"
node_processing_parameters = [3]
consensus_protocol = "IBFT"

## Fault settings
num_faults = 3
fault_type = "UR"
fault_parameters = []

# Network settings
## Switch settings
switch_processing_distribution = "degen"
switch_processing_parameters = [0]
message_channel_success_rate = 1

network_type = "Clique"
network_parameters = []

In [ ]:
VALIDATOR_RESULTS_FILEPATH = "../json"
RESULTS_DIRECTORY = "json_n{num_nodes}_btl{base_time_limit:.1f}_{node_dist}_{node_params}_{topology}_{topo_params}_{switch_dist}_{switch_params}" +  \
                    "_{protocol}_{num_faults}_{fault_type}_{fault_params}"

In [ ]:
def process_results(results_dic: dict): 
    processed_results_dic = {}
    state_names = None
    for var in results_dic.keys():
        temp_dic = results_dic[var].copy()
        if state_names == None:
            state_names = temp_dic[STATE_NAMES_KEY].keys()
        for key in temp_dic[STATE_NAMES_KEY].keys():
            temp_dic[key] = temp_dic[STATE_NAMES_KEY][key]
        del temp_dic[STATE_NAMES_KEY]
        processed_results_dic[var] = temp_dic
    return (processed_results_dic, state_names) 

def write_str_to_file(file_string: str, filename: str) -> None:
    with open(filename, "w") as file:
        file.write(file_string)

def run_and_save(run_config_dic: str, output_directory: str) -> None:
    config_filename = "config.json"
    write_str_to_file(run_config_dic, config_filename)

    argument = "py/" + config_filename
    ! (cd "../" && gradlew run --args={argument})
    copy_tree(VALIDATOR_RESULTS_FILEPATH, output_directory)    

def construct_results_directory(num_nodes: int, base_time_limit: float, 
                                node_processing_distribution: str, node_processing_parameters: List[float],
                                topology: str, topo_params: List[int], switch_processing_distribution: str, switch_processing_parameters: List[float], 
                                protocol: str, num_faults: int, fault_type: str, fault_params: List[int]) -> str:
    return RESULTS_DIRECTORY.format(num_nodes=num_nodes,  base_time_limit=base_time_limit, 
                                    node_dist=node_processing_distribution, node_params=node_processing_parameters, 
                                    topology=topology, topo_params=topo_params, switch_dist=switch_processing_distribution, switch_params=switch_processing_parameters, 
                                    protocol=protocol, num_faults=num_faults, fault_type=fault_type, fault_params=fault_params)


In [ ]:
results_dic = {}
# for base_time_limit in [10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20, 20.5, 21, 21.5, 22, 22.5, 23, 24, 25, 26, 27, 28, 29, 30] + list(range(31, 51)): 
for base_time_limit in [20, 20.5, 21, 21.5, 22, 22.5, 23, 24, 25, 26, 27, 28, 29, 30] + list(range(31, 71)): 
# for base_time_limit in range(31, 51):
# for base_time_limit in [5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5]:
# for base_time_limit in [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5]:
# for num_nodes in [4, 8, 16, 24, 32, 48, 64]:
    json_obj = create_run_config_json(num_runs, starting_seed, seed_multiplier,
                                      num_nodes, num_consensus, base_time_limit, 
                                      node_processing_distribution, node_processing_parameters, 
                                      consensus_protocol, num_faults, fault_type, fault_parameters,
                                      switch_processing_distribution, switch_processing_parameters, 
                                      message_channel_success_rate, network_type, network_parameters)
    run_and_save(json_obj, construct_results_directory(num_nodes, float(base_time_limit), 
                                                       node_processing_distribution, node_processing_parameters, 
                                                       network_type.lower(), network_parameters, 
                                                       switch_processing_distribution, switch_processing_parameters, 
                                                       consensus_protocol.lower(),
                                                       num_faults, fault_type, fault_parameters))


In [ ]:
RESULTS_FOLDER_REGEX = r'json_n(.+)_btl(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)_(.+)'

def get_num_nodes(filename: str) -> int:
    return int(re.match(RESULTS_FOLDER_REGEX, filename).group(1))

def get_btl(filename: str) -> float:
    return float(re.match(RESULTS_FOLDER_REGEX, filename).group(2))

def get_topology(filename: str) -> str:
    return re.match(RESULTS_FOLDER_REGEX, filename).group(5)

def get_protocol(filename: str) -> str:
    return re.match(RESULTS_FOLDER_REGEX, filename).group(9)

def get_num_faults(filename: str) -> int:
    return int(re.match(RESULTS_FOLDER_REGEX, filename).group(10))

def get_node_distribution(filename: str) -> str:
    return re.match(RESULTS_FOLDER_REGEX, filename).group(3)

In [ ]:
RESULTS_VALIDATOR_FILENAME = "validator_results.json"
RESULTS_FOLDER = "results"
FASTEST_MESSAGE_MAP = "fastestMessageCountMap"
REMAINDER_MESSAGE_MAP = "remainderMessageCountMap"
FASTEST_TIME_MAP = "fastestStateTimeMap"
REMAINDER_TIME_MAP = "remainderStateTimeMap"
PREPARED = "PREPARED"
PREPREPARED = "PREPREPARED"
COMMIT = "COMMIT"
SYNC = "SYNC"
ROUND_CHANGE = "ROUND_CHANGE"
TOTAL_TIME_KEY = "t_total_fastest"
RC_PROB = "RC_PROB"
NEW_ROUND = "NEW_ROUND"
PRE_PREPARED = "PRE_PREPARED"
LAMBDA_FASTEST = "lambda_fastest"
L_FASTEST = "L_fastest"
L_REMAINDER = "L_remainder"

NEW_VIEW = "NEW_VIEW"
PREPARE = "PREPARE"
PRE_COMMIT = "PRE_COMMIT"
DECIDE = "DECIDE"
COMMIT = "COMMIT"

IBFT_STATES = [NEW_ROUND, PRE_PREPARED, PREPARED, ROUND_CHANGE]
HS_STATES = [PREPARE, PRE_COMMIT, COMMIT, DECIDE]
PROTOCOL_NAME_STATE_MAP = {"hs": HS_STATES, "ibft": IBFT_STATES}

In [ ]:
lst_num_faults = [0, 1, 2, 3]
def get_num_faults_data(num_faults: int, name: str) -> pd.Series:
    results_lst = os.listdir("results/")
    jsons = []
    index = []
    t_total = []
    states = PROTOCOL_NAME_STATE_MAP[consensus_protocol.lower()] 
    fastest_state_times = {state : [] for state in states}
    remainder_state_times = {state : [] for state in states}
    fastest_message_arrival_rates = []

    fastest_message_queue_lengths = []
    remainder_message_queue_lengths = []
    for result_filename in results_lst:
        matcher = re.match(RESULTS_FOLDER_REGEX, result_filename)
        if matcher == None: 
            continue
        run_num_nodes = get_num_nodes(result_filename) 
        run_base_time_limit = get_btl(result_filename) 
        run_topology = get_topology(result_filename) 
        run_protocol = get_protocol(result_filename) 
        run_num_faults = get_num_faults(result_filename)
        run_dist = get_node_distribution(result_filename)
 
        if run_protocol != consensus_protocol.lower() or run_num_nodes != num_nodes or run_num_faults != num_faults or run_dist != node_processing_distribution:
            continue
 
        index.append(run_base_time_limit)
        with open(os.path.join(RESULTS_FOLDER, result_filename, RESULTS_VALIDATOR_FILENAME), "r") as json_result:
            result_json = json.load(json_result)
            jsons.append(result_json)
            t_total.append(result_json[TOTAL_TIME_KEY])
            fastest_message_arrival_rates.append(result_json[LAMBDA_FASTEST])
            fastest_message_queue_lengths.append(result_json[L_FASTEST])
            remainder_message_queue_lengths.append(result_json[L_REMAINDER])
            for state in states:
                fastest_state_times[state].append(result_json[FASTEST_TIME_MAP][state])
                remainder_state_times[state].append(result_json[REMAINDER_TIME_MAP][state])

    return pd.Series(t_total, name=name, index=index)

df = pd.DataFrame({"0_fault": get_num_faults_data(0, "0_fault"), "1_fault": get_num_faults_data(1, "1_fault"), "2_fault": get_num_faults_data(2, "0_fault"), 
                   "3_fault": get_num_faults_data(3, "3_fault")})
df = df.sort_index().interpolate(method="linear")
df.plot(grid=True, style=".-", xlabel="base_time_limit", ylabel="time_to_consensus", title=consensus_protocol + " simulation")
plt.show()
# df = pd.DataFrame(fastest_state_times, index=index)
# # df["t_total"] = t_total
# # df.sort_index(inplace=True)
# # df.plot(grid=True, style=".-")

In [ ]:
def hs_time(n, t, mu, te2e):
    max_f = (n - 1) // 3
    p = gamma.cdf(t, 4 * n + 3 - max_f, scale=(1/mu))
    # print(p)
    t_fail = (1 - p) * t 
    # p2 = gamma.cdf(2 * t, 4 * n + 3  - max_f, scale=(1/mu))
    # t_fail += (1 - p) * (1 - p2) * (2 * t)
    t_succeed = 1 * te2e
    # p3 = gamma.cdf(4 * t, 4 * n + 3 - f, scale=(1/mu))
    # t_fail += (1 - p) * (1 - p2) * (1 - p3) * (4 * t)
    t_total = t_fail + t_succeed
    return t_total

def hs_time_fault(n, t, rate, f):
    max_f = (n - 1) // 3
    m = (4 * n + 4 - max_f - 3 * f) 
    te2e = m / rate 

    p_fault = f / n
    t_penalty_first_fault = 0 
    for i in range(1, f + 1):
        t_penalty_first_fault += p_fault ** i * t * (2 ** (i - 1)) 

    p = gamma.cdf(t, m - 1, scale=(1/rate))
    t_fail = (1 - p) * (t + t_penalty_first_fault * 2)
    t_total = t_fail * (n - f) / n + te2e 

    # print(t_total, t_fail, t_succeed, t_penalty_first_fault)
    return t_total + t_penalty_first_fault

df["prediction"] = df.index.map(lambda t: hs_time_fault(num_nodes, t, node_processing_parameters[0], num_faults))
# df["prediction"] = df.index.map(lambda t: full_estimate(num_nodes, num_faults, 1/3, 1/3, t))
df.plot(style=".-", figsize=(10, 5), grid=True, xlabel="base_time_limit", ylabel="time per consensus")
# Testing recommendation
def hs_reco(n, f, mu, sigma):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    sigma_m = np.sqrt(m) * sigma 
    return (m * mu + 3 * sigma_m) 

test_num_nodes = [16, 32, 64, 128, 256, 512, 1024, 2048]
test_mu = 1 / 3
test_num_faults = [2]

def range_max_estimate(n, mu):
    max_f = (n - 1) // 3
    m = 4 * n - max_f 
    return int(m * mu * 1.75)

def range_min_estimate(n, mu):
    # return int(range_max_estimate(n, mu) / 1.75 / 2)
    return 0

def get_minima(n, mu, frac):
    max_f = (n - 1) // 3
    f = int(frac * max_f)
    top = range_max_estimate(n, mu)
    bot = range_min_estimate(n, mu)
    if frac == 0:
        return hs_time_fault(n, top, 1/mu, f) 
    rg = [bot + (top - bot) / 1000 * i for i in range(1000)] 
    lst = [hs_time_fault(n, t, 1/mu, max_f * frac // 2) for t in rg] 
    s = pd.Series(lst, index=rg)
    temp = s[(s < s.shift(1)) & (s < s.shift(-1))]
    return temp.iloc[0]

def get_num_faults_from_frac(n, frac):
    return int((n - 1) // 3 * frac / 2)

def hs_estimate_left_line(n, f, mu, t):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    te2e = m * mu
    # gradient = n / (n - 2 * f) 
    # return te2e + gradient * t
    one = te2e + sum((f / n) ** (i - 1) * t * 2 ** (i - 1) for i in range(1, f + 1))
    one *= f / n
    two = te2e + sum((f / n) ** i * (t * 2 ** i) for i in range(f + 1))
    two *= (n - f) / n
    return one + two


def hs_estimate_right_line(n, f, mu, t):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    mean = m * mu
    gradient = f / (n - 2 * f)
    return mean + gradient * t

def full_estimate(n, f, mu, sigma, t):
    max_f = (n - 1) // 3
    m = 4 * n - max_f - 3 * f
    mean = m * mu
    sigma_m = np.sqrt(m) * sigma
    bot_boundary = mean - 2 * sigma_m
    top_boundary = mean + 2 * sigma_m
    y1 = hs_estimate_left_line(n, f, mu, bot_boundary)
    y2 = hs_estimate_right_line(n, f, mu, top_boundary)
    if t > top_boundary:
        return hs_estimate_right_line(n, f, mu, t)
    elif t < bot_boundary:
        return hs_estimate_left_line(n, f, mu, t)
    else:
        gradient = (y2 - y1) / (top_boundary - bot_boundary)
        intercept = y2 - gradient * top_boundary
        return gradient * t + intercept


# dic_model = {j: [get_minima(n, test_mu, j) for n in test_num_nodes] for j in test_num_faults}
# dic_guess = {j: [hs_time_fault(n, hs_reco(n, get_num_faults_from_frac(n, j), test_mu, test_mu), 1/test_mu, get_num_faults_from_frac(n, j)) 
#                  for n in test_num_nodes] for j in test_num_faults}
# df_model = pd.DataFrame(dic_model, index=test_num_nodes)
# df_guess = pd.DataFrame(dic_guess, index=test_num_nodes)
# (df_guess - df_model) / df_model

plot_num_nodes = 2048
rg = range(range_min_estimate(plot_num_nodes, test_mu), range_max_estimate(plot_num_nodes, test_mu))
dic = {str(plot_num_nodes) + "_" + str(j): [hs_time_fault(plot_num_nodes, t, 1/test_mu, get_num_faults_from_frac(plot_num_nodes, j)) 
                               for t in range(range_min_estimate(plot_num_nodes, test_mu), range_max_estimate(plot_num_nodes, test_mu))]
                               for j in test_num_faults}
dic["test"] = [full_estimate(plot_num_nodes, get_num_faults_from_frac(plot_num_nodes, 2), test_mu, test_mu, t) for t in rg]
pd.DataFrame(dic, index=rg).plot( figsize=(15, 10), grid=True, ylabel="time to consensus", xlabel="base time limit")
[plt.axvline(hs_reco(plot_num_nodes, get_num_faults_from_frac(plot_num_nodes, f), test_mu, test_mu)) for f in test_num_faults]


In [ ]:
### IBFT WORK DO NOT TOUCH
round_change_probs = np.array(map(lambda dic: min(dic[FASTEST_MESSAGE_MAP][PREPREPARED] - 1, 1), jsons))
consensus_times = np.array(map(lambda dic: dic[TOTAL_TIME_KEY], jsons))

dic = {RC_PROB: round_change_probs, TOTAL_TIME_KEY: consensus_times, ROUND_CHANGE: fastest_state_times[ROUND_CHANGE], "COMBINED": fastest_state_times["COMBINED"],"z_value": ((np.array(index) - 10) / 2.5)}

df = pd.DataFrame(dic, index=index)
df.sort_index(inplace=True)
df["RC_PROB"].plot(style=".-", grid=True, xlabel="base time limit", ylabel="probability of round change")
df["z_value"].apply(lambda x: 1 - norm.cdf(x)).plot()
###
def time3(t):
    def helper(t, message_penalty):
        if t >= 20:
            return 12 
        elif t <= 4:
            return t + helper(2 * t, max(0, message_penalty - int(t * 3)) + 16)
        p = norm.cdf((t - message_penalty / 3 - 10) / 2.5)
        return (1 - p) * (8 + t + helper(2 * t, 5)) + p * min(12, t)
    return helper(t, 0) - 2.7 + calc(t, 3)

###
# pd.Series(range(1, 40)).map(time3).plot()
df["prediction"] = df.index.map(time3)
df[[TOTAL_TIME_KEY, "prediction"]].plot(grid=True, style=".-")

### break
rho = 2.67 / 3
def pi(i):
    return (1 - rho) * pow(rho, i)

def calc(t, processing_rate):
    i = 0
    E = 0
    cdf = 0
    while i / processing_rate <= t:
        E += (i / processing_rate) * pi(i)
        cdf += pi(i)
        i += 1
    E += (1 - cdf) * t 
    p = (1 - norm.cdf((t - 12)))
    p2 = 4 * p / 16
    E = E * (1 - p2) + p2 * t
    return E

pd.DataFrame({ROUND_CHANGE: fastest_state_times[NEW_ROUND], "TEST": map(lambda x : calc(x, 3), index)}, index=index).sort_index().plot(grid=True, style=".-")